# Data Ops Challenge Main Analysis File
In this file, I'll be using a Jupyter notebook to produce the core visualizations that will supplement the presentation and be used for answering the provided guiding questions.

## Loading, Adjusting, and Merging the Data

In [1]:
#imports
import sqlite3
import pandas as pd
import plotly.graph_objects as plt
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

In [2]:
#first, i'm going to initially connect to the database
conn = sqlite3.connect("challenge.db")

#now i'll load in each table as a dataframe
cities_df = pd.read_sql("SELECT * FROM five_hundred_cities;", conn)
access_df = pd.read_sql("SELECT * FROM access;", conn)
variablelist_df = pd.read_sql("SELECT * FROM variable_list", conn)

In [3]:
#now, when reading through the tables, i saw that the FIPS value for the access table was NOT the standard 5 digits for some
#of the codes. after looking up the FIPS codes, i found out that any FIPS code that originally had a "0" at the beginning
#was now missing a 0 at the beginning. if it had a non-zero value at the beginning, it still had that value. so i'll be
#adding in a zero at the beginning of each FIPS code that was supposed to have a 0 in the access table
#this was actually the case in the 500 cities dataset as well, with the PlaceFIPS and TractFIPS codes so i fixed that too.
#i've included a print before and after to show that this was the case.

print(cities_df["TractFIPS"].head())
cities_df["TractFIPS"] = cities_df["TractFIPS"].astype(str).str.zfill(11).str[:5]
print(cities_df["TractFIPS"].head())

print(access_df["FIPS"].head(10))
access_df["FIPS"] = access_df["FIPS"].astype(str).str.zfill(5)
print(access_df["FIPS"].head(10))

0    1073000100
1    1073000300
2    1073000400
3    1073000500
4    1073000700
Name: TractFIPS, dtype: int64
0    01073
1    01073
2    01073
3    01073
4    01073
Name: TractFIPS, dtype: object
0    1001
1    1003
2    1005
3    1007
4    1009
5    1011
6    1013
7    1015
8    1017
9    1019
Name: FIPS, dtype: int64
0    01001
1    01003
2    01005
3    01007
4    01009
5    01011
6    01013
7    01015
8    01017
9    01019
Name: FIPS, dtype: object


In [4]:
#now, to make analysis easier since we want to consider both factors from the FDA food atlas and from the
#CDC 500 cities dataset, we're gonna merge the two tables. as i specified in the README.md, i noted that the
#first 5 digits of the tract FIPS code for the 500 cities dataset (not the city FIPS code for the 500 cities dataset)
#and the first 5 digits of the FIPS code for the FDA food atlas match 
#we also will extract lat/lon from geolocation "(lat, lon)" strings for visualization purposes later (specifically visualization 1)
cities_df[["lat","lon"]] = cities_df["Geolocation"].str.strip("()").str.split(",", expand=True)
cities_df["lat"] = cities_df["lat"].astype(float)
cities_df["lon"] = cities_df["lon"].astype(float)
merged = cities_df.merge(
    access_df,
    left_on="TractFIPS",   #county FIPS from cities in tract FIPS
    right_on="FIPS",       #padded county FIPS from the access table
    how="inner"            #only keep matches
)
print(merged.columns.tolist()) #making sure all the columns merged

['index_x', 'StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID', 'Population2010', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI', 'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev', 'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI', 'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev', 'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI', 'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev', 'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI', 'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev', 'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI', 'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev', 'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI', 'DIABETES_CrudePrev', 'DIABETES_Crude95CI', 'HIGHCHOL_CrudePrev', 'HIGHCHOL_Crude95CI', 'KIDNEY_CrudePrev', 'KIDNEY_Crude95CI', 'LPA_CrudePrev', 'LPA_Crude95CI', 'MAMMOUSE_CrudePrev', 'MAMMOUSE_Crude95CI', 'MHLTH_CrudePrev', 'MHLTH_Crude95CI',

In [5]:
#here, i'll be cleaning up the data for NaN/missing data by just dropping rows that include missing data.
#i'll print out the total number of rows, and the number of rows with at least one missing value so it can be
#seen that the number of rows dropped is not a large portion of the total number of rows we have.
total_rows = len(merged)
print(f"total rows in merged dataset: {total_rows}")

#these are columns that i use for visualization later on
key_cols = [
    "PCT_LACCESS_POP15", "COPD_CrudePrev", "CASTHMA_CrudePrev",
    "DIABETES_CrudePrev", "ARTHRITIS_CrudePrev", "BPHIGH_CrudePrev",
    "COREM_CrudePrev", "COREW_CrudePrev",
    "Population2010",
    "LACCESS_LOWI15", "LACCESS_HHNV15", "LACCESS_SNAP15",
    "LACCESS_CHILD15", "LACCESS_SENIORS15", "LACCESS_WHITE15",
    "LACCESS_BLACK15", "LACCESS_HISP15", "LACCESS_NHASIAN15",
    "LACCESS_NHNA15", "LACCESS_NHPI15", "LACCESS_MULTIR15"
]

merged = merged.replace([-9999, -8888, -1, "N/A", "NA", "NaN"], np.nan)
rows_with_missing = merged[key_cols].isna().any(axis=1).sum()
print(f"number of rows with at least one missing value in key columns: {rows_with_missing}")

#dropping rows where key identifiers or metrics are missing
merged = merged.dropna(subset=key_cols)
total_after_merge = len(merged)
print(f"total rows after dropping missing value rows: {total_after_merge}")

total rows in merged dataset: 27210
number of rows with at least one missing value in key columns: 190
total rows after dropping missing value rows: 27020


## Visualization 1 (the where)
This visualization aims to help supplement the guiding question **"where should we deploy a food access program?"**

We can find this out by first considering that the only table we have available for analysis from the FDA Food Access dataset is the "ACCESS" sub-table, which contains information about access and proximity to grocery stores with the following subcategories within that category:
- overall (population with low-access)
- household resources (low income & low access, households with no car & low access, SNAP households & low access)
- demographics (children, seniors, hispanic ethnicity, asian, american indian or alaska native, asian, hawaiian or pacific islander, multiracial with low access)

We should also consider that for the 500 cities dataset, though some of the columns regarding different health conditions and their prevalences only consider elderly populations (>= 65 years old or 50-74 years old) or only adult men or only adult women, generally it doesn't account for factors such as household resources or demographics but rather just for a general age range (18-64), it looks at "what is the crude prevalence of this health condition?"

Since we want to consider data from both datasets when we are considering where to deploy a food access program, we'll only consider the overall access to grocery stores (general population with low-access) from the FDA Food Access dataset so as to match the generality of the 500 cities dataset.

Though we have past data that we can use and a percent change we can consider with access to food in specific areas (change from 2010-2015), I will only be using the percentage of population impacted by low access to food in specific areas from the most recent data (2015) so that we can target where need is highest now. For an example on why we want to target this to answer the question of where we should deploy a food access program now, we can consider that if we have a town with a large change of overall low access to grocery stores of 3% to 6% from 2010 to 2015 and another town with no change of overall low access to grocery stores of 25% to 25% from 2010 to 2015, we would want to target the latter town for food access programs since, although they have didn't change, they are experiencing more food access challenges with more of their population now.

In a 2024 study by Gregory and Jones on health challenges related to household food security (https://www.ers.FDA.gov/data-products/charts-of-note/chart-detail?chartId=108211), it was found that 5 chronic diseases increased their prevalence as household food security worsened. These diseases were **hypertension, arthritis, diabetes, asthma, and COPD**. The age range included was ages 19-64 working adults, with compared household food security ranging from high, marginal, low, and very low. I'll be using these 5 diseases as the target ones to look for in the 500 cities dataset related to low food access in the FDA food atlas dataset when trying to find locations to deploy a food access program.

In [6]:
#what are the columns that I want to analyze for this? (note: hypertension = long-term high blood pressure)
    #county, city, state for the actual location (County, PlaceName, State)
    #PCT_LACCESS_POP15 from the FDA food atlas dataset -> indicates population, low access to store (%), 2015
    #COPD_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of chronic obstructive pulmonary disease among adults aged >=18 years
    #CASTHMA_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of current asthma among adults aged >=18 years
    #DIABETES_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of diagnosed diabetes among adults aged >=18 years
    #ARTHRITIS_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of arthritis among adults aged >=18 years
    #BPHIGH_CrudePrev from the 500 cities dataset -> indicates the model-based estimate for crude prevalence of high blood pressure among adults aged >=18 years

#first, what we'll do is calculate a "priority score" for each county that we have available in our merged dataset.
#this will be calculated as the following:
    #mean(disease scores for copd, asthma, diabetes, arthritis, and hypertension) * low access to store % for that county
#the higher the score -> the more priority we should designate to that county to designate resources.
#why does this work? -> multiplication highlights counties with both high food access problems AND high disease prevalence

#creating a copy of the merged df for adding in the calculatuions of disease avg and priority score for each location
vis1_data = merged.copy() 
vis1_data["disease_avg"] = vis1_data[["COPD_CrudePrev", "CASTHMA_CrudePrev", "DIABETES_CrudePrev","ARTHRITIS_CrudePrev", "BPHIGH_CrudePrev"]].mean(axis=1)
vis1_data["priority_score"] = vis1_data["PCT_LACCESS_POP15"] * vis1_data["disease_avg"]

#now we aggregate the data on a county-level and state-level for the purposes of the visualization.
county_data = vis1_data.groupby(["FIPS", "County", "State"]).agg(
    priority_score=("priority_score", "mean"), #average priority score across all tracts in that county
    lat=("lat", "mean"), #average the centroid coordinates
    lon=("lon", "mean") #average the centroid coordinates
).reset_index() #make it into a normal df

state_data = county_data.groupby("State").agg(
    priority_score=("priority_score", "mean") #average priority score across all counties in that state
).reset_index() #make it into a normal df

#we also normalize the data to be on a 0-100 scale for the priority score so the visualization is easier to understand
min_val = state_data["priority_score"].min()
max_val = state_data["priority_score"].max()
state_data["priority_score_norm"] = ((state_data["priority_score"] - min_val) / (max_val - min_val) * 100)

#we get the top 10 highest priority score counties overall (not per state) to display on the map
#as the areas we should focus on
top10 = county_data.nlargest(10, "priority_score").copy()
#need to normalize data to be on 0-100 scale for the top 10 counties as well (like we did at the state level)
min_val_c = county_data["priority_score"].min()
max_val_c = county_data["priority_score"].max()
top10["priority_score_norm"] = ((top10["priority_score"] - min_val_c) / (max_val_c - min_val_c) * 100)
top10 = top10.sort_values("priority_score", ascending=False).reset_index(drop=True) #sort the values in descending order

#adding labels to the top 10 counties: rank, county, state and priority score
top10["RankLabel"] = (
    (top10.index+1).astype(str) + ". " +
    top10["County"] + ", " +
    top10["State"] +
    " (" + top10["priority_score_norm"].round(1).astype(str) + ")"
)

In [7]:
#here we'll actually generate the figure!
fig = plt.Figure()
#the background will be a state-level heatmap chloropleth
fig.add_trace(plt.Choropleth(
    locations=state_data["State"], #which geographic places are being colored
    z=state_data["priority_score_norm"], #numeric values used to determine color for each state (normalized priority score)
    locationmode="USA-states", #how to interpret locations field
    colorscale="RdBu_r",   #color scheme to shade the states
    colorbar_title="Priority Score (0–100)", #color bar title
    colorbar=dict(
        len=0.6,           #scales down the colorbar length (default is 1)
        thickness=15,      
        x=1,             
        y=0.5               
    ),
    showscale=True
))

#this overlays the top 10 counties we found before with markers and labels
for i, row in top10.iterrows():
    rank = i + 1
    fig.add_trace(plt.Scattergeo(
        lon=[row["lon"]],
        lat=[row["lat"]],
        mode="markers+text",
        name=f"{rank}. {row['County']}, {row['State']}",  # Legend text only
        text=f"#{rank}",                                  # Rank label on map
        textposition="top center",
        marker=dict(
            size=10,
            color="black",
            symbol="circle",
            line=dict(width=1, color="white")
        ),
        showlegend=True,
        legendgroup=f"group{rank}",
        textfont=dict(size=12, color="black"),
        # ✅ This hides the dot marker in the legend
        marker_symbol=None,
        hoverinfo="text"
    ))
fig.update_geos(scope="usa") #tells plotly to show the usa
#this part gives the plot a title, updates how big it'll be, and also tells it to not give it a legend (it'll overlap w/ the bar)
fig.update_layout(
    title=dict(
        text="State-Level Food Access Program Priority<br>with Top 10 Counties Highlighted",
        y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top",
        font=dict(size=18)
    ),
    height=750,
    showlegend=True,
    legend=dict(
        title="Top 10 Counties",
        x=-0.12,
        y=0.5,
        bgcolor="rgba(255,255,255,0.6)",
        bordercolor="gray",
        borderwidth=0.5,
        font=dict(size=11)
    )
)
fig.show() #show the figure


## Visualization 2 & 3 (the who)
This visualization aims to supplement the guiding questions **"how many people will be included? how many might be successfully engaged?"** and **"which subgroup of the population might benefit the most from the program?"**

In order to answer these questions, we can first and foremost take into account that we already have the top 10 counties & states where we should be deploying food access programs. According to the above visualization and the data given, those are:
1. Hamilton, TN
2. Dougherty, GA
3. Genesee, MI
4. Richmond, GA
5. Plymouth, MA
6. Brevard, FL
7. Osage, OK
8. Lake, IN
9. Harrison, MS
10. Wichita, TX

So, what we know for these areas is that their calculated "priority score", which is a multiplication of the percentage of the general population's low access to store as of 2015 and the crude prevalence of 5 chronic diseases who are most affected by limited food access in >= 18 year olds, is significantly high enough to be in this top 10. Therefore, these counties should be where the resources are deployed.

In order to first answer the question, **how many people will be included?**, we should first consider the fact that the client is a national Medicare Advantage program, which is a form of health insurance. Therefore, any programs that would be deployed to help relieve food access would likely be targeting existing members of health insurance programs. We should also consider the fact that Medicare is a program generally meant for people 65 years or older, and some people under 65 with certain disabilities or conditions. Therefore, since the general population we are targeting for these food access programs would need to be on health insurance and 65+. What we can do is use a supplementary dataset called the "County Demographics CSV File" from the CORGIS Project by Ryan Whitcomb, Joung Min Choi, and Bo Guan (https://corgis-edu.github.io/corgis/csv/county_demographics/). In this dataset, we are given information "about counties in the United States from 2010 through 2019 through the United States Census Bureau," and this includes a variable that is an "estimated percentage of population whose ages are equal or greater than 65 years old" for various County, State places in the United States. We can use this percentage, coupled with the "Population2010" census number value for each County, State place given from the 500 cities dataset, to calculate a relatively accurate estimate of the number of 65+ year old adults in that given area. Specifically, since we only need to consider the top 10 counties that we found earlier, we can just sort that CSV file for those specific top 10 counties and use those values. Values are denoted as -1 in this dataset if they are not available.

Next, to answer the question **how many might be successfully engaged?** we should again consider the fact that the client we are presenting to is a National Medicare Advantage program, and we should likely look at the population that is 65+ and on health insurance. For engagement purposes, we can specifically use the variables "COREM_CrudePrev" and "COREW_CrudePrev" which provide:
- Model-based estimate for crude prevalence of older adult men aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening, and
- Model-based estimate for crude prevalence of older adult women aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening, and Mammogram

These variables provide numbers we want to use for total population that would be engaged in these top 10 priority cities because they are in the target age range (65+) for Medicare Advantage programs, actively use healthcare services (and therefore likely have some form of health insurance), and are more likely to a health outreach/food access program like the one proposed for this.

Last, to answer the question **which subgroup of the population might benefit the most from the program?**, we can use the FDA food atlas dataset as it contains more specific information about demographics and the like for which demographic groups are disproportionately affected within these top 10 counties, and therefore most likely to benefit from interventions. To be more clear with the figure, and also since within the dataset itself groups it this way, we will split up the visualization for this part into household-based subgroups affected and demographics-based subgroups affected by food access challenges. As a note, it's important to consider the fact that, when looking at this visualization, people can overlap in subgroup membership (e.g. be a part of a SNAP household, be a senior, and be white).

In [8]:
#getting information from the corgis csv file, making sure to only include the columns we need
#and clean the names
demographic_data = pd.read_csv('county_demographics.csv')
demographic_data = demographic_data.rename(columns={
    "County": "County",
    "State": "State",
    "Age.Percent 65 and Older": "Percent65plus"
})
#removing any extra words that might come after the county name in the csv file
def clean_county_name(name):
    return (name.replace(" County", "")
            .replace(" Parish", "")
            .replace(" City", "")
            .replace(" Borough", "")
            .strip()
            .title()
    )
demographic_data["County"] = demographic_data["County"].apply(clean_county_name)
demographic_data["State"] = demographic_data["State"].str.strip().str.upper()

#making sure that the percent 65 plus has no -1 values (missing values)
before_rows = len(demographic_data)
demographic_data = demographic_data[demographic_data["Percent65plus"] != -1].dropna(subset=["Percent65plus"])
after_rows = len(demographic_data)

print(f"{before_rows - after_rows} rows with missing -1 or NaN Percent65plus values")
print(f"remaining rows: {after_rows}")

0 rows with missing -1 or NaN Percent65plus values
remaining rows: 3139


In [9]:
#for the subgroup part we have the following in the FDA food atlas for subgroups/demographics:
#household resources (low income & low access, households with no car & low access, SNAP households & low access)
#demographics (children, seniors, hispanic ethnicity, asian, american indian or alaska native, asian, hawaiian or pacific islander, multiracial with low access)
#we do have some missing information for 2010 values, so we will have to use the 2015 values for these demographics.
#how can we calculate impact for these subgroups? -> well, in the FDA food atlas each of these percentages for these
#demographics indicates the number that had a LOW access to a grocery store
#so we could take all of the values within these top 10 counties (2015 counts)
#and find out which groups are most affected by low access to a grocery store in these 10 counties and therefore would
#benefit most from the program. so for each subgroup, we calculate estimated affected individuals

#we create a new copy of the merged dataframe for the purposes for getting all of the columns we need (that aren't in the
#top10 dataframe we created earlier for visualization1)
vis2_data = merged.copy()
#getting the same sorted, merged by county data with inclusion of population, household and demographic low access to grocery store %
#we do population-weighted mean for corem and corew calculation b/c we want to ensure that the percentages are as accurate as
#possible for the county overall
#we do .iloc[0] to just get the first value since in the FDA food atlas dataset we dont have duplicate counties but in the 500 cities one
#we do so we dont want to sum up the counts for the FDA food atlas variables
vis2_county = vis2_data.groupby(["FIPS", "County", "State"]).apply(
    lambda x: pd.Series({
        'population': x['Population2010'].sum(),
        'corem': (x['COREM_CrudePrev'] * x['Population2010']).sum() / x['Population2010'].sum(),
        'corew': (x['COREW_CrudePrev'] * x['Population2010']).sum() / x['Population2010'].sum(),
        'low_income': x['LACCESS_LOWI15'].iloc[0],
        'households_no_car': x['LACCESS_HHNV15'].iloc[0],
        'SNAP_household': x['LACCESS_SNAP15'].iloc[0],
        'children': x['LACCESS_CHILD15'].iloc[0],
        'seniors': x['LACCESS_SENIORS15'].iloc[0],
        'white': x['LACCESS_WHITE15'].iloc[0],
        'black': x['LACCESS_BLACK15'].iloc[0],
        'hispanic': x['LACCESS_HISP15'].iloc[0],
        'asian': x['LACCESS_NHASIAN15'].iloc[0],
        'american_indian_or_alaska_native': x['LACCESS_NHNA15'].iloc[0],
        'hawaiian_or_pacific_islander': x['LACCESS_NHPI15'].iloc[0],
        'multiracial': x['LACCESS_MULTIR15'].iloc[0]
    }),
    include_groups=False
).reset_index()

#cleaning formatting for merging
vis2_county["County"] = vis2_county["County"].str.strip().str.title()
vis2_county["State"] = vis2_county["State"].str.strip().str.upper()

#merging with CORGIS
vis2_county = vis2_county.merge(
    demographic_data[["County", "State", "Percent65plus"]],
    on=["County", "State"],
    how="left"
)
#calculate actual number of 65+ population
vis2_county["population_65plus"] = vis2_county["population"] * vis2_county["Percent65plus"] / 100

#we get only the top 10 counties (same as the ones plotted in visualization 1)
vis2_county = vis2_county[vis2_county["FIPS"].isin(top10["FIPS"])].copy()

#get the engaged population with using both COREM and COREW as representative of older adults who are actively using healthcare services
vis2_county["engaged"] = vis2_county["population_65plus"] * ((vis2_county["corem"] + vis2_county["corew"]) / 200)

#adding rank info to the vis2_county dataframe so we can order the barplot
rank_map = dict(zip(top10["FIPS"], range(1, len(top10)+1)))
vis2_county["Rank"] = vis2_county["FIPS"].map(rank_map)
vis2_county = vis2_county.sort_values("Rank")
county_labels = vis2_county["Rank"].astype(str) + ". " + vis2_county["County"] + ", " + vis2_county["State"]

#calculating subgroup affected numbers -> first we get all the column names
subgroup_cols = [
    "low_income", "households_no_car", "SNAP_household",
    "children", "seniors", "white", "black", "hispanic", "asian",
    "american_indian_or_alaska_native", "hawaiian_or_pacific_islander", "multiracial"
]

#melting for plotting purposes so now we have one row per county per subgroup (instead of one row per county)
subgroup_data = vis2_county.melt(
    id_vars=["County", "State", "population", "engaged"], #these dont get melted
    value_vars=subgroup_cols, #these do get melted
    var_name="Subgroup",
    value_name="Affected"
)

#for plotting
subgroup_data["CountyState"] = subgroup_data["County"] + ", " + subgroup_data["State"]
label_map = {
    "low_income": "Low Income & Low Access",
    "households_no_car": "Households w/ No Car & Low Access",
    "SNAP_household": "SNAP Households & Low Access",
    "children": "Children & Low Access",
    "seniors": "Seniors & Low Access",
    "white": "White & Low Access",
    "black": "Black & Low Access",
    "hispanic": "Hispanic & Low Access",
    "asian": "Asian & Low Access",
    "american_indian_or_alaska_native": "American Indian/Alaska Native & Low Access",
    "hawaiian_or_pacific_islander": "Hawaiian/Pacific Islander & Low Access",
    "multiracial": "Multiracial & Low Access"
}
subgroup_data["Subgroup"] = subgroup_data["Subgroup"].map(label_map)


In [10]:
#visualization 1 -> bar chart for total vs engaged population (65+ y/o)
fig1 = plt.Figure()
fig1.add_trace(
    plt.Bar(
        x=county_labels,
        y=vis2_county["population_65plus"],
        marker_color="#0277C8",
        name="Total 65+ Population"
    )
)
fig1.add_trace(
    plt.Bar(
        x=county_labels,
        y=vis2_county["engaged"],
        marker_color="#33AD7A",
        name="Healthcare-Engaged 65+"
    )
)
fig1.update_layout(
    barmode="group",
    height=500,
    width=900,
    title=dict(
        text="Total vs Engaged Population (Top 10 Counties, 65+ years old)",
        y=0.86,
        x=0.5,
        xanchor="center",
        yanchor="top",
        font=dict(size=18)
    ),
    legend=dict(
        orientation="h",
        x=0.02,
        y=1,
        font=dict(size=10)
    ),
    xaxis_tickangle=-45,
    xaxis_title="County, State",
    yaxis_title="Population (People)"
)
fig1.show()


In [ ]:
#visualization 2: subgroup analysis for the top 10 counties, where we group the subgroups into household demographics
#and other demographics
subgroup_summary = subgroup_data.groupby("Subgroup")["Affected"].sum().reset_index()
household_groups = subgroup_summary[subgroup_summary["Subgroup"].isin([
    "Low Income & Low Access",
    "Households w/ No Car & Low Access",
    "SNAP Households & Low Access"
])].copy()
household_groups["Percent"] = household_groups["Affected"] / household_groups["Affected"].sum() * 100

#horizontal bar chart for household-based graph
fig_household = px.bar(
    household_groups.sort_values("Percent", ascending=True),
    x="Percent",
    y="Subgroup",
    orientation="h",
    color="Subgroup",
    color_discrete_sequence=["#0277C8", "#33AD7A", "#304C64"],
    title="Household-Based Subgroups Affected by Low Food Access"
)
fig_household.update_layout(
    height=400,
    width=700,
    xaxis_title="Percent of Total Household-Based Population",
    yaxis_title="",
    legend_title="Subgroup",
    showlegend=False,
    xaxis=dict(ticksuffix="%"),
    font=dict(size=12)
)
fig_household.show()


demographic_groups = subgroup_summary[subgroup_summary["Subgroup"].isin([
    "Children & Low Access",
    "Seniors & Low Access",
    "White & Low Access",
    "Black & Low Access",
    "Hispanic & Low Access",
    "Asian & Low Access",
    "American Indian/Alaska Native & Low Access",
    "Hawaiian/Pacific Islander & Low Access",
    "Multiracial & Low Access"
])].copy()

#sorting by affected count and grouping everything after the top 5 into "other" for better visualization
demographic_groups = demographic_groups.sort_values("Affected", ascending=False)
top5 = demographic_groups.head(5).copy()
other_sum = demographic_groups.iloc[5:]["Affected"].sum()

# Add "Other" as a combined category
top5.loc[len(top5)] = ["Other", other_sum]
top5["Percent"] = top5["Affected"] / top5["Affected"].sum() * 100

#donut chart for demographic-based (top 5 + other)
fig_demo = px.pie(
    top5,
    names="Subgroup",
    values="Percent",
    hole=0.45,
    color_discrete_sequence=px.colors.diverging.RdBu_r,
    title="Demographic-Based Subgroups Affected (Top 5 + Other)"
)
fig_demo.update_traces(
    textinfo="label+percent",
    insidetextorientation="horizontal"
)
fig_demo.update_layout(
    height=450,
    width=700,
    showlegend=False,
    title_y=0.9,
    font=dict(size=12)
)
fig_demo.show()


## Putting it all together: The Impact
This section aims to supplement the guiding question **"what is the projected impact of this program?"**

Given all of the information we collected above, we can form a pretty good picture of what the impact of such a food access program deployed by this National Medicare Advantage plan might be.

First, from the first section, we know which top 10 counties which have the highest priority score (due to having the highest rates of low-access to grocery stores AND highest rates of the 5 diseases most affected by low food access). 

From these top 10 counties, we also know generally a total number of adults that are Medicare-eligible (adults aged 65+), and from these adults which are most actively engaged in healthcare services from the second section. Additionally, from the section section, we know which subgroups within these counties (not specifically all 65+) are most affected within these 10 counties due to low food access and therefore have a general idea of which subgroups (demographic and household-based) to prioritize for outreach.

Given all this information, we can first calculate an engagement rate percentage as the total 65+ adults that are engaged in healthcare services divided by the total number of 65+ adults within these 10 counties times 100. Then, we can conservatively estimate that of those that are engaged, ~50% of those will be reached and will be engaged for this food access program.

In [12]:
total_65plus = vis2_county["population_65plus"].sum()
total_engaged = vis2_county["engaged"].sum()
engagement_rate = (total_engaged / total_65plus) * 100
print("engagement rate: " + str(int(engagement_rate)) + "%")

estimated_reach = 0.5 * total_engaged
print(f"estimated reach: {int(estimated_reach):,} members")

engagement rate: 27%
estimated reach: 28,311 members


Then, what we can do is estimate potential health improvement given the average mean chronic disease prevalence within the top 10 counties for the 5 diseases that are most affected by low food access. If we assume that we have a 5% relative reduction in chronic disease healthcare costs among participants due to better food access, we can estimate:

In [13]:
disease_avg_mean = vis1_data["disease_avg"].mean() / 100
potential_health_improvement = estimated_reach * 0.05 * disease_avg_mean
print(f"potential impact: improving management for ~{int(potential_health_improvement)} members with chronic diseases")

potential impact: improving management for ~224 members with chronic diseases


Now, what we can do is translate the reduction to cost impact for the Medicare Advantage plan. 

The 5 chronic diseases that are most affected by low food access are:
- high blood pressure (hypertension)
- COPD
- asthma
- diabetes
- arthritis

The following studies & articles I found covered the healthcare costs of these 5 chronic diseases (per person, per year):
- Wang, Yu et al. “Health Care Expenditures and Use Associated with Hypertension Among U.S. Adults.” American journal of preventive medicine vol. 67,6 (2024): 820-831. doi:10.1016/j.amepre.2024.07.005 -> $2,759
- Mannino, David M., et al. “National and Local Direct Medical Cost Burden of COPD in the United States From 2016 to 2019 and Projections Through 2029.” CHEST, vol. 165, no. 5, May 2024, pp. 1093–106. journal.chestnet.org, https://doi.org/10.1016/j.chest.2023.11.040. -> $1,795
- https://aafa.org/advocacy/key-issues/access-to-health-care/cost-of-asthma-on-society/ -> $3,728
- Olga A. Khavjou, Minglu Sun, Sophia R. D’Angelo, Simon J. Neuwahl, Thomas J. Hoerger, Pyone Cho, Kristopher Myers, Ping Zhang; Economic Costs Attributed to Diagnosed Diabetes in Each U.S. State and the District of Columbia: 2021. Diabetes Care 20 June 2025; 48 (7): 1164–1171. https://doi.org/10.2337/dc24-0832 -> $8,474 (Medicare cost specifically)
- Zhao, X., et al. “Clinical, Humanistic, and Economic Burden of Osteoarthritis among Noninstitutionalized Adults in the United States.” Osteoarthritis and Cartilage, vol. 27, no. 11, Nov. 2019, pp. 1618–26. www.oarsijournal.com, https://doi.org/10.1016/j.joca.2019.07.002. -> $1,778

So, given these numbers, we can take an average of these per person, per year costs for these chronic diseases and multiply it by the above number of chronic disease cases we can prevent with this food access program to find the cost avoidance benefit of this program.

In [14]:
average_cost = (2759 + 1795 + 3728 + 8474 + 1778) / 5
overall_cost_benefit = potential_health_improvement * average_cost
print(f"estimated annual cost avoidance: ${int(overall_cost_benefit):,}")
print(f"this represents potential savings of ${int(overall_cost_benefit/estimated_reach):,} per engaged member")

estimated annual cost avoidance: $830,559
this represents potential savings of $29 per engaged member
